<a id="setup"></a>
##  Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).


### Install and import dependecies

In [2]:
import os, getpass
import pandas as pd
import humanize
import random
from typing import Optional, Any, Iterable, List
from dotenv import load_dotenv
load_dotenv('../.env')

True

### watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud user API key. For details, see <a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank" rel="noopener no referrer">documentation</a>.

In [3]:
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": os.getenv('api_key', None)
}

### Defining the project id
The API requires project id that provides the context for the call. We will obtain the id from the project in which this notebook runs. Otherwise, please provide the project id.

**Hint**: You can find the `project_id` as follows. Open the prompt lab in watsonx.ai. At the very top of the UI, there will be `Projects / <project name> /`. Click on the `<project name>` link. Then get the `project_id` from Project's Manage tab (Project -> Manage -> General -> Details).


In [4]:
try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Please enter your project_id (hit enter): ")

<a id="data"></a>
## Data (test) loading

Download the test dataset. This dataset is used to calculate the metrics score for selected model, defined prompts and parameters.

In [5]:
filename_test = './questions_test.csv'
filename_train =  './questions_train.csv'

test_data = pd.read_csv(filename_test)
train_data = pd.read_csv(filename_train)

Inspect data sample

In [6]:
train_data.head()

,qid,question,answers
0,1961,where does diffusion occur in the excretory sy...,diffusion
1,7528,when did the us join world war one,"April 6 , 1917"
2,8685,who played wilma in the movie the flintstones,Elizabeth Perkins
3,6716,when was the office of the vice president created,1787
4,2916,where does carbon fixation occur in c4 plants,in the mesophyll cells


### Build up knowledge base

The current state-of-the-art in RAG is to create dense vector representations of the knowledge base in order to calculate the semantic similarity to a given user query.

We can generate dense vector representations using embedding models. In this notebook, we use <a href="https://www.sbert.net/" target="_blank" rel="noopener no referrer">Sentence Transformers</a> <a href="https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2" target="_blank" rel="noopener no referrer">all-MiniLM-L6-v2</a> to embed both the knowledge base passages and user queries. `all-MiniLM-L6-v2` is a performant open-source model that is small enough to run locally.

A vector database is optimized for dense vector indexing and retrieval. This notebook uses <a href="https://python.langchain.com/docs/integrations/vectorstores/elasticsearch#basic-example" target="_blank" rel="noopener no referrer">Elasticsearch</a>, a distributed, RESTful search and analytics engine, capable of performing both vector and lexical search. It is built on top of the Apache Lucene library, which offers good speed and performance with all-MiniLM-L6-v2 embedding model.

The dataset we are using is already split into self-contained passages that can be ingested by Elasticsearch. 

The size of each passage is limited by the embedding model's context window (which is 256 tokens for `all-MiniLM-L6-v2`).

### Load knowledge base documents

Load set of documents used further to build knowledge base. 

In [7]:
knowledge_base_dir = "./knowledge_base"

In [8]:
my_path = f"{os.getcwd()}/knowledge_base"
if not os.path.isdir(my_path):
   os.makedirs(my_path)

In [9]:
documents = pd.read_csv(f"{knowledge_base_dir}/psgs.tsv", sep='\t', header=0)
documents['indextext'] = documents['title'].astype(str) + "\n" + documents['text']
documents = documents[:1000]

### Create an embedding function

Note that you can feed a custom embedding function to be used by Elasticsearch. The performance of Elasticsearch may differ depending on the embedding model used.

In [10]:
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.embeddings.base import Embeddings

emb_func = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\admin\OneDrive - Bittek Soluciones Tecnológicas\FORMACION\generative_ai\generative_ai\watsonx_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\admin\OneDrive - Bittek Soluciones Tecnológicas\FORMACION\generative_ai\generative_ai\watsonx_env\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\admin\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-t

<a id="models"></a>
## Foundation Models on watsonx

### Defining model
You need to specify `model_id` that will be used for inferencing:

In [11]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes

model_id = ModelTypes.FLAN_UL2

### Defining the model parameters
We need to provide a set of model parameters that will influence the result:

In [12]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams
from ibm_watson_machine_learning.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.GREEDY,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 50
}

### Initialize the `Model` class.

In [13]:
from ibm_watson_machine_learning.foundation_models import Model

watsonx_granite = Model(
    model_id=model_id.value,
    credentials=credentials,
    project_id=project_id,
    params=parameters
).to_langchain()

## Set up Elasticsearch Trial account
Go to <a href="https://cloud.elastic.co/login?redirectTo=%2Fhome"> Elasticsearch </a>  page, and create an account (recommended with google or microsoft).


<a id="elastic_conn"></a>

We'll use the Cloud ID to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.
To find the password for the `elastic` user, go to https://cloud.elastic.co/deployments and select your deployment. Then on the left-hand sided menu select the `Security` settings.
Click on the `Reset password` button and copy the generated password.


The following cell retrieves the Elasticsearch Cloud ID and password for the `elastic` user from the environment if available and prompts you otherwise.

In [14]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
try:
    es_cloud_id = os.environ["ELASTIC_CLOUD_ID"]
except KeyError:
    es_cloud_id = input("Please enter your Elasticsearch Cloud ID (hit enter): ")


try:
    es_password = os.environ["ELASTIC_PASSWORD"]
except KeyError:
    es_password = input("Please enter your Elasticsearch PASSWORD (hit enter): ")

<a id="elasticsearchstore"></a>
## Set up ElasticsearchStore connector from Langchain


We first create a regular Elasticsearch Python client connection. Then we pass it into LangChain's ElasticsearchStore wrapper together with the WatsonX model based embedding function.

Consult the LangChain documentation For more information about <a href="https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html" target="_blank" rel="noopener no referrer">ElasticsearchStore</a> connector.

In [15]:
from langchain_elasticsearch import ElasticsearchStore
from elasticsearch import Elasticsearch

# Create the client instance
es_connection = Elasticsearch(
    cloud_id=es_cloud_id,
    basic_auth=("elastic", es_password)
)

# Successful response!
es_connection.info()


knowledge_base = ElasticsearchStore(es_connection=es_connection,
                                    index_name="test_index",
                                    embedding=emb_func,
                                    strategy=ElasticsearchStore.ApproxRetrievalStrategy(),
                                    distance_strategy="DOT_PRODUCT")


<a id="elasticsearchstore_index"></a>
### Embed and index documents with Elasticsearch

**Note: Could take several minutes if you don't have pre-built indices**

In [16]:
if es_connection.indices.exists(index="test_index"):
    es_connection.indices.delete(index="test_index")
_ = knowledge_base.add_texts(texts=documents.indextext.tolist(),
                             metadatas=[{'title': title, 'id': doc_id}
                                for (title, doc_id) in
                                zip(documents.title, documents.id)],  # filter on these!
                             index_name="test_index",
                             ids=[str(i) for i in documents.id]  # unique for each doc
                            )

Let's take a look in Elasticsearch what the LangChain wrapper has created. First we display the newly created index ("tables" in Elasticsearch are always called "index"). Note the field `vector` of type `dense_vector` with `dot_product` similarity.

In [17]:
dict(es_connection.indices.get(index="test_index"))

{'test_index': {'aliases': {},
  'mappings': {'properties': {'metadata': {'properties': {'id': {'type': 'long'},
      'title': {'type': 'text',
       'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}},
    'text': {'type': 'text',
     'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}},
    'vector': {'type': 'dense_vector',
     'dims': 384,
     'index': True,
     'similarity': 'dot_product'}}},
  'settings': {'index': {'routing': {'allocation': {'include': {'_tier_preference': 'data_content'}}},
    'number_of_shards': '1',
    'provided_name': 'test_index',
    'creation_date': '1716473289555',
    'number_of_replicas': '1',
    'uuid': '9O7O6U_gTVaai8Fv5oKmiA',
    'version': {'created': '8503000'}}}}}

Verify the number of documents loaded into the Elasticsearch index.

In [18]:
doc_count = es_connection.count(index='test_index')["count"]
doc_count

1000

Let's retrieve a random document as a sample. Note the embedding in the vector field, that was generated with the WatsonX embedding model.

In [19]:
dict(es_connection.get(index="test_index", id=random.randint(0, len(documents)-1)))

{'_index': 'test_index',
 '_id': '350',
 '_version': 1,
 '_seq_no': 349,
 '_primary_term': 1,
 'found': True,
 '_source': {'text': "Gold standard\nused in coins worth less than $1 ( fractional currency ) . With the resumption of convertibility on June 30 , 1879 the government again paid its debts in gold , accepted greenbacks for customs and redeemed greenbacks on demand in gold . Greenbacks were therefore perfect substitutes for gold coins . During the latter part of the nineteenth century the use of silver and a return to the bimetallic standard were recurrent political issues , raised especially by William Jennings Bryan , the People 's Party and the Free Silver movement . In 1900 the gold dollar was declared the standard unit of account and a gold reserve for government issued paper notes was established . Greenbacks , silver certificates , and silver dollars continued to be legal tender , all redeemable in gold . Fluctuations in the US gold stock , 1862 -- 1877 US gold stock 1862 

Display the total size and indexing time of the new index in Elasticsearch.

In [20]:
index_stats = es_connection.indices.stats(index="test_index").get('_all').get('primaries')
print("Index size:    " + humanize.naturalsize(index_stats.get('store').get('size_in_bytes')))
print("Indexing time: " + humanize.precisedelta(index_stats.get('indexing').get('index_time_in_millis')/1000, minimum_unit='minutes'))


Index size:    4.7 MB
Indexing time: 0 minutes


<a id="predict"></a>
## Generate a retrieval-augmented response to a question

`RetrievalQA` is a chain to do question answering.

**Hint:** To use Chain interface from LangChain with watsonx.ai models you must call `model.to_langchain()` method. 

It returns `WatsonxLLM` wrapper compatible with LangChain CustomLLM specification.

### Select questions

The prompts we will use to test the RAG flow

In [21]:
questions_and_answers = {
            'names of founding fathers of the united states?': "Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton",
            'who played in the super bowl in 2013?': 'Baltimore Ravens::San Francisco 49ers',
            'when did bucharest become the capital of romania?': '1862'
}

### Retrieve relevant context

Fetch paragraphs similar to the question

In [22]:
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(llm=watsonx_granite, chain_type="stuff",verbose=True, retriever=knowledge_base.as_retriever(), return_source_documents=True)

In [23]:
results = []


for question in questions_and_answers.keys():

    result = qa.invoke({'query': question})

    print("result: ", result)
    results.append( result)



> Entering new RetrievalQA chain...

> Finished chain.
result:  {'query': 'names of founding fathers of the united states?', 'result': 'John Adams , Benjamin Franklin , Alexander Hamilton , John Jay , Thomas Jefferson , James Madison , and George Washington', 'source_documents': [Document(page_content="Founding Fathers of the United States\n^ Burstein , Andrew . `` Politics and Personalities : Garry Wills takes a new look at a forgotten founder , slavery and the shaping of America '' , Chicago Tribune ( November 09 , 2003 ) : `` Forgotten founders such as Pickering and Morris made as many waves as those whose faces stare out from our currency . '' ^ Jump up to : Rafael , Ray . The Complete Idiot 's Guide to the Founding Fathers : And the Birth of Our Nation ( Penguin , 2011 ) . Jump up ^ `` Founding Fathers : Virginia '' . FindLaw Constitutional Law Center . 2008 . Retrieved 2008 - 11 - 14 . Jump up ^ Schwartz , Laurens R. Jews and the American Revolution : Haym Solomon and Others , 

Get the set of chunks for one of the questions.

In [24]:
for idx, result in enumerate(results):
    print("=========")
    print("Question = ", result['query'])
    print("Answer = ", result['result'])
    print("Expected Answer(s) (may not be appear with exact wording in the dataset) = ", questions_and_answers[result['query']])
    print("\n")
    print("Source documents:")
    print(*(x.page_content for x in result['source_documents']), sep='\n')
    print("\n")
    

Question =  names of founding fathers of the united states?
Answer =  John Adams , Benjamin Franklin , Alexander Hamilton , John Jay , Thomas Jefferson , James Madison , and George Washington
Expected Answer(s) (may not be appear with exact wording in the dataset) =  Thomas Jefferson::James Madison::John Jay::George Washington::John Adams::Benjamin Franklin::Alexander Hamilton


Source documents:
Founding Fathers of the United States
^ Burstein , Andrew . `` Politics and Personalities : Garry Wills takes a new look at a forgotten founder , slavery and the shaping of America '' , Chicago Tribune ( November 09 , 2003 ) : `` Forgotten founders such as Pickering and Morris made as many waves as those whose faces stare out from our currency . '' ^ Jump up to : Rafael , Ray . The Complete Idiot 's Guide to the Founding Fathers : And the Birth of Our Nation ( Penguin , 2011 ) . Jump up ^ `` Founding Fathers : Virginia '' . FindLaw Constitutional Law Center . 2008 . Retrieved 2008 - 11 - 14 . 

---

Copyright © 2023 IBM. This notebook and its source code are released under the terms of the MIT License.